In [ ]:
#@title Connecting to google drive (Before Running Main Code Connect to Drive)🥇
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive/')

# # Authenticate Google Drive API
from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
auth.authenticate_user() 
creds, _ = default() 
gc = gspread.authorize(creds) 
import gspread_dataframe as gd



import pandas as pd
import shutil
import os  
import datetime
import glob
import re


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#@title Main Code

path = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Thrive/Thrive reports/Summary' # copy paste your path





# ***************************************************************************** MAIN CODE **************************************************************************************
count=0
data=pd.DataFrame()

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            read_csv=pd.read_csv(file_path,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],skiprows=7)

            read_csv2=pd.read_csv(file_path,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'])

            read_csv.insert(1,'Period From',read_csv2.iloc[1]['Column2'],allow_duplicates=False)
            read_csv.insert(2,'Period To',read_csv2.iloc[2]['Column2'],allow_duplicates=False)
            read_csv.insert(3,'Res_name',read_csv2.iloc[0]['Column2'],allow_duplicates=False)
            read_csv.insert(0,'Source_file',os.path.basename(file_path),allow_duplicates=False)
            data=pd.concat([data,read_csv])
            count+=1
print('Total Files - ',count)


#@title Droping Null Rows
data.dropna(0,how='all',subset=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],inplace=True)


#@title Droping rows and Renaming the cols

data.dropna(0,how='any',subset=['Column1'],inplace=True)

r=	['Column1','Column2'	,	'Column3'	,'Column4', 'Column5','Column6']
d=['Particular','Delivery Orders',	'Take-Away Orders',	'Dine In Orders'	,'Cancelled Orders'	,'Total Orders']
rename_cols=dict(zip(r,d))
#rename_cols
data.rename(columns=rename_cols,inplace=True)


#@title alpha serials for particulars
	
alpha_serials={'Number Of Orders':	'A',
              'Item Sub-Total':	'B',
              'Packaging Or Service Charges':	'C',
              'Delivery Charges'	:'D',
              'Discounts':	'E',
              'Net Bill Value'	:'F',
              'GST Charged to Customer':	'G',
              'Total Customer Payable':	'H',
              'Hashtag Loyalty Service Fees - Direct':	'I',
              'Razorpay Transaction Fee':	'J',
              'Delivery Fee':	'K',
              'Commission - Consumer - New User':'L',
              'Commission - Consumer - Repeat User':'M',
              'Total GST over fees'	:'N',
              'Total Service Fees':	'O',
              'GST Deducted':	'P',
              'TCS Deducted'	:'Q',
              'TDS Deducted':	'R',
              'Net Payable'	:'S'}

#@title add year
data['Year']=data['Period To'].str.split('-',expand=True)[2]


#@title Adding Month and report periods, droping cols


data['Frm day']=data['Period From'].str.split('-',expand=True)[0]
data['Frm Mon']=data['Period From'].str.split('-',expand=True)[1]
data['Frm Mon name']=pd.to_datetime(data['Frm Mon'],format='%m').dt.month_name().str.slice(stop=3)


data['To day']=data['Period To'].str.split('-',expand=True)[0]
data['To Mon']=data['Period To'].str.split('-',expand=True)[1]
data['To Mon name']=pd.to_datetime(data['To Mon'],format='%m').dt.month_name().str.slice(stop=3)



data['New Period From']= data['Frm day'] + " " + data['Frm Mon name'].astype(str)
data['New Period To']= data['To day'] + " " + data['To Mon name'].astype(str)


data['Report Period']=data['New Period From']+"-"+data['New Period To']


data.drop(columns=['Frm Mon','Frm day','To day','To Mon','To Mon name','New Period From','New Period To'],inplace=True)
data.rename(columns={'Frm Mon name':'Month'},inplace=True)


#@title Serial no. fr months
#data['Month'].unique()
months=['Mar','Apr','May','Jun', 'Jul','Aug','Sep','Oct','Nov', 'Dec','Jan']


data.head(2)

In [ ]:
#@title go

thriv_sales=data[data['Particular'].isin(['Net Bill Value','TDS Deducted'])]

brand_code={"Chinese Chaska"	:"CC",
            "Mann Food Court"	:"MM",
            "Dilli Darbar"	:"DD",
            "Roj Ka Khana"	:"RR"}

thriv_sales['Brand_code']=thriv_sales['Res_name'].map(brand_code)

l1=thriv_sales['Report Period'].str.split("-",expand=True)[0].str.replace(" ","",regex=True)
l2=thriv_sales['Report Period'].str.split("-",expand=True)[1].str.replace(" ","",regex=True)
thriv_sales['Invoice Number']='TH'+thriv_sales['Brand_code']+l1+l2+thriv_sales['Year'].str.slice(start=2,stop=4)


thriv_sales['Invoice Date']=thriv_sales['Report Period'].str.split("-",expand=True)[1]+" "+thriv_sales['Year']



thriv_sales['Particular'].replace('Net Bill Value','Food Sales',inplace=True)


In [ ]:
#@title go2

thriv_sales["Customer Name"]	=	'Thrive'+" "+thriv_sales['Res_name']
thriv_sales["Invoice Status"]	=	"Overdue"
thriv_sales["GST Treatment"]	=	"Consumer"
thriv_sales["Place of Supply"]	=	"KA"
thriv_sales["Currency Code"]	=	"INR"
thriv_sales["Exchange Rate"]	=	"1"
thriv_sales["Account"]	=	"Sales"
thriv_sales.rename(columns={'Particular':"Item Name"},inplace=True)
thriv_sales["Item Type"]	=	"Goods"
thriv_sales["Quantity"]	=	"1"
thriv_sales["Usage unit"]	=	"Count"
thriv_sales["Is Inclusive Tax"]	=	"Tax"
thriv_sales["Item Tax"]	=	"GST5"
thriv_sales["Item Tax Type"]	=	"Tax Group"
thriv_sales["Item Tax %"]	=	"0%"
thriv_sales["Supply Type"]	=	"Exempted"
thriv_sales["Discount Type"]	=	"entity_level"
thriv_sales["Adjustment"]	=	"Adjustment"
thriv_sales.rename(columns={'Total Orders':"Item Price"},inplace=True)

thriv_sales["E-Commerce Operator Name"]	=	"Hashtag Loyalty Private Limited"
thriv_sales["E-Commerce Operator GSTIN"]	=	"27AADCH9016M1Z9"
thriv_sales["TDS Name"]	=	"Rx-TDS on E-commerce Transactions"
thriv_sales["TDS Percentage"]	=	"1"
thriv_sales["TDS Section Code"]	=	"194O"


# condition = (thriv_sales['Item Name'] != 'Food Sales') & (thriv_sales['Item Price'] != 0.0)
# thriv_sales.loc[condition, 'Item Price'] = thriv_sales.loc[condition, 'Item Price']
thriv_sales.drop(columns=['Res_name','Delivery Orders', 'Take-Away Orders', 'Dine In Orders','Cancelled Orders','Brand_code'],inplace=True)
mask = thriv_sales['Item Name'] == 'TDS Deducted'

# Assign the 'Item Price' values from rows where 'Item Name' is 'TDS' to the 'TDS Amount' column
thriv_sales.loc[mask, 'TDS Amount'] = thriv_sales.loc[mask, 'Item Price']

thriv_sales=thriv_sales[::-1]
thriv_sales['TDS Amount']=thriv_sales['TDS Amount'].fillna(method='ffill')
thriv_sales=thriv_sales[::-1]


thriv_sales=thriv_sales[~ (thriv_sales['Item Name']=='TDS Deducted')]

In [ ]:
import gspread_dataframe as gd

wc=gc.open_by_key('13yBVsGXL_p1p2Yi5HXvav8DU7z5jYHWRb3ch-4LCWko')
wx=wc.worksheet('sheet1')

#gd.set_with_dataframe(wx,thriv_sales)